<a href="https://colab.research.google.com/github/dawidmakowski-git/tensorflow_tutorials/blob/master/Custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf

print(tf.test.is_gpu_available())

True


# Layers: common sets of useful operations

In [13]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [14]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [15]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_5/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.58827597, -0.43166047, -0.41523564, -0.15648386,  0.6056027 ,
         -0.04036039,  0.08446485,  0.25619543,  0.23893428, -0.5861916 ],
        [-0.22778162, -0.50348544,  0.32469803,  0.4823119 ,  0.6098409 ,
          0.58519024, -0.43309373,  0.10918039, -0.27657792, -0.21299389],
        [-0.5963945 ,  0.5251481 , -0.24190244,  0.41253167,  0.4915716 ,
         -0.18617153,  0.15095228,  0.21551329, -0.17238441,  0.4813853 ],
        [-0.56056774, -0.52324605, -0.01322705, -0.48314562, -0.13474262,
          0.48939663,  0.5478981 , -0.03434843,  0.4883421 , -0.27410737],
        [-0.5717095 , -0.39423415, -0.22948194, -0.0204401 , -0.12578994,
          0.2639742 , -0.24774477,  0.6127655 ,  0.2934696 ,  0.42205554]],
       dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [18]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_5/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.58827597, -0.43166047, -0.41523564, -0.15648386,  0.6056027 ,
         -0.04036039,  0.08446485,  0.25619543,  0.23893428, -0.5861916 ],
        [-0.22778162, -0.50348544,  0.32469803,  0.4823119 ,  0.6098409 ,
          0.58519024, -0.43309373,  0.10918039, -0.27657792, -0.21299389],
        [-0.5963945 ,  0.5251481 , -0.24190244,  0.41253167,  0.4915716 ,
         -0.18617153,  0.15095228,  0.21551329, -0.17238441,  0.4813853 ],
        [-0.56056774, -0.52324605, -0.01322705, -0.48314562, -0.13474262,
          0.48939663,  0.5478981 , -0.03434843,  0.4883421 , -0.27410737],
        [-0.5717095 , -0.39423415, -0.22948194, -0.0204401 , -0.12578994,
          0.2639742 , -0.24774477,  0.6127655 ,  0.2934696 ,  0.42205554]],
       dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

# Implementing custom layers

In [19]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


## **Models: Composing layers**

In [21]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

_ = block(tf.zeros([1, 2, 3, 3])) 

block.layers

In [22]:
len(block.variables)

18

In [23]:
block.summary()

Model: "resnet_identity_block_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_3 (Batch multiple                  4         
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_4 (Batch multiple                  8         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_5 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
______________________________________________

In [24]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [25]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_6 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_7 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_8 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________